In [1]:
!pip install xlrd
!pip install openpyxl

import xlrd
import numpy as np
import string
import requests
from bs4 import BeautifulSoup
import json
import openpyxl

In [2]:
import pandas as pd
print("pandas version: ", pd.__version__)
pd.set_option('display.max_row',1000)
pd.set_option('display.max_columns', 100)

pandas version:  1.0.5


In [3]:
# 아메리카노만 
def find_americano(menu_str):
    menu_list = menu_str.split('|')
    for menu in menu_list:
        if "메리카노" in menu:
            return menu

In [4]:
def dobong_cafe():
    
    list = ['도봉1동','도봉2동','방학1동','방학2동','방학3동',
            '쌍문1동','쌍문2동','쌍문3동','쌍문4동','창1동','창2동','창3동','창4동','창5동']
    
    result = pd.DataFrame()
    is_first = True
    
    for dong in list:
        base_url = 'https://map.naver.com/v5/api/search'
        params = {
        'caller':'pcweb',
        'query': '도봉구'  + dong +'카페',
        'type':'all',
        'searchCoord':'126.88415050506593;37.48030724508602',
        'page':'1',
        'displayCount':'100',
        'isPlaceRecommendationReplace':'true',
        'lang':'ko'
        }
        headers = {
            'user-agent' :  'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.105 Safari/537.36'
        }

        resp = requests.get(base_url,params=params,headers=headers)
        data = json.loads(resp.text)

        # # 카페명, 도로명, 메뉴명 추출한 리스트 만들기
        
        name_menu_list = []    
        try:
            data_list = data['result']['place']['list']        
        except TypeError:
            print('해당 동에 카페가 없습니다.')
        for i in data_list:
            name_menu_list.append({
                'Name':i['name'],
                'Address':i['roadAddress'],
                'Menu':i['menuInfo']
            })
        
        #  메뉴명 None값 있는 행 삭제
        
        del_none = []
        for i in name_menu_list:
            if i['Menu']!= None:
                del_none.append(i)
            
         # 데이터 프레임 만들기

        cafe_americano_df = pd.DataFrame(del_none)

        # 아메리카노만 
        
        def find_americano(menu_str):
            menu_list = menu_str.split('|')
            for menu in menu_list:
                if "메리카노" in menu:
                    return menu
                
        cafe_americano_df['Americano'] = cafe_americano_df['Menu'].apply(find_americano) 
        cafe_americano = cafe_americano_df.dropna(axis=0)

        # strip
        cafe_americano['Americano Price1'] = cafe_americano['Americano'].apply(lambda x:x.strip())
        
        #  가격만
        cafe_americano['Americano Price2'] = cafe_americano['Americano Price1'].apply(lambda x:x.split(' ')[-1])
        
        # '변동가격(업주문의)'제거
        
        #         for i in cafe_americano.iter_rows():
        #             pass
        def make_price(x):
            try:
                if '변동가격' in x:
                    return None
                else:
                    return int(x.replace(',', ''))
            except:
                return None
        cafe_americano['Americano Price'] = cafe_americano['Americano Price2'].apply(
            make_price
        )
       
        # '도봉구'인 카페만 출력
        
        addr_filtered = cafe_americano[cafe_americano['Address'].str.contains("도봉구")]
        
        # Menu,Americano Price1,Americano 삭제        
        #addr_filtered.drop('Menu',axis=1,inplace=True)
        addr_filtered.drop('Americano Price1',axis=1,inplace=True)
        addr_filtered.drop('Americano',axis=1,inplace=True)
        addr_filtered.drop('Americano Price2',axis=1,inplace=True)
        
        # 병합
        if is_first:
            result = addr_filtered
            is_first = False
        else:
            result.append(addr_filtered, ignore_index=True)
            
        # excel 저장
        result.to_excel('도봉구카페.xlsx')
        
        # 출력
        
        
    return result
               


In [5]:
a = dobong_cafe()

<ipython-input-4-7f0f22874d06>:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cafe_americano['Americano Price1'] = cafe_americano['Americano'].apply(lambda x:x.strip())
<ipython-input-4-7f0f22874d06>:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cafe_americano['Americano Price2'] = cafe_americano['Americano Price1'].apply(lambda x:x.split(' ')[-1])
<ipython-input-4-7f0f22874d06>:82: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,

In [6]:
a

,Name,Address,Menu,Americano Price
0,카페152길,서울특별시 도봉구 도봉로152길 29-3,"아메리카노 3,000 | 오리진 치즈케익 1호 30,000 | 플레인롤케익 18,0...",3000
1,Cafe1253,서울특별시 도봉구 도봉로180길 6 태봉운동시설,"아메리카노 5,000",5000
3,도봉산커피,서울특별시 도봉구 도봉산4가길 4,"에스프레소 2,500 | 아메리카노 3,000 | 카페라떼 3,500 | 카푸치노 ...",3000
4,산과 산 사이 커피로드,서울특별시 도봉구 도봉로180가길 96,"아메리카노 2,800 | 카페라떼 3,800 | 카푸치노 3,800 | 허브티 3,...",2800
6,CAFE187,서울특별시 도봉구 도당로29길 65,"아메리카노 2,500 | 카페라떼 3,500 | 바닐라라떼 4,000 | 카페모카 ...",2500
7,이디야커피 도봉산점,서울특별시 도봉구 도봉산길 18 동서빌딩 1층,"아메리카노 3,200",3200
9,주인장이볶은커피,서울특별시 도봉구 마들로 772 1층 주인장이볶은커피,"원두 판매 5,500 | 7종 블랜딩 아메리카노 3,000 | 스모킹 라떼 3,50...",3000
10,CAFE BEGINNING,서울특별시 도봉구 마들로 764 보성빌딩,"아메리카노 3,500",3500
11,제이네가게,서울특별시 도봉구 도봉로167길 7 행운빌라 104호,"제이의 브루잉 - HOT 4,000 | 제이의 브루잉 - ICE 4,300 | 라떼...",3000
14,V45 카페,서울특별시 도봉구 도봉로164길 43,"아메리카노 3,000 | 카페 라떼 4,000 | 스무디 4,000 | 생맥주 3,000",3000


In [ ]:
'변동가격(업주문의)'
ValueError

In [49]:
def dobong_cafe():
    
    list = ['도봉1동','도봉2동','방학1동','방학2동','방학3동',
            '쌍문1동','쌍문2동','쌍문3동','쌍문4동','창1동','창2동','창3동','창4동','창5동']
    
    
    for dong in list:
        base_url = 'https://map.naver.com/v5/api/search'
        params = {
        'caller':'pcweb',
        'query': '도봉구'  + dong +'카페',
        'type':'all',
        'searchCoord':'126.88415050506593;37.48030724508602',
        'page':'1',
        'displayCount':'100',
        'isPlaceRecommendationReplace':'true',
        'lang':'ko'
        }
        headers = {
            'user-agent' :  'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.105 Safari/537.36'
        }

        resp = requests.get(base_url,params=params,headers=headers)
        data = json.loads(resp.text)

        # # 카페명, 도로명, 메뉴명 추출한 리스트 만들기
        
        name_menu_list = []    
        try:
            data_list = data['result']['place']['list']        
        except TypeError:
            print('해당 동에 카페가 없습니다.')
        for i in data_list:
            name_menu_list.append({
                'Name':i['name'],
                'Address':i['roadAddress'],
                'Menu':i['menuInfo']
            })
        
        #  메뉴명 None값 있는 행 삭제

        del_none = []
        for i in name_menu_list:
            if i['Menu']!= None:
                del_none.append(i)
            
         # 데이터 프레임 만들기

        cafe_americano_df = pd.DataFrame(del_none)

        # 아메리카노만         
        cafe_americano_df['Americano'] = cafe_americano_df['Menu'].apply(find_americano) 
        cafe_americano = cafe_americano_df.dropna(axis=0)

        # strip
        cafe_americano['Americano Price1'] = cafe_americano['Americano'].apply(lambda x:x.strip())
        
        #  가격만
        cafe_americano['Americano Price2'] = cafe_americano['Americano Price1'].apply(lambda x:x.split(' ')[-1])
        
        # '변동가격(업주문의)'제거
        if '변동가격(업주문의)' in cafe_americano['Americano Price2']:
            
        else:
            cafe_americano['Americano Price'] = cafe_americano['Americano Price2'].str.replace(',','').astype('int')
        
        # Americano Price에서 ','지우기 / 정수형으로 바꾸기        
        
        #cafe_americano['Americano Price'] = cafe_americano['Americano Price2'].str.replace(',','').astype('int')
        
    
        # '도봉구'인 카페만 출력
        
        addr_filtered = cafe_americano[cafe_americano['Address'].str.contains("도봉구")]
        
        # Menu,Americano Price1,Americano 삭제        
        #addr_filtered.drop('Menu',axis=1,inplace=True)
        addr_filtered.drop('Americano Price1',axis=1,inplace=True)
        addr_filtered.drop('Americano',axis=1,inplace=True)
        addr_filtered.drop('Americano Price2',axis=1,inplace=True)
        
        # 병합
        result = pd.DataFrame()
        for i in range(len(list)):
            result = result.append(addr_filtered, ignore_index=True)
        
        # excel 저장
        result.to_excel('도봉구카페.xlsx')
        
        # 출력
        
        
    return result
               


In [50]:
 dobong_cafe()

<ipython-input-49-8244facd69a7>:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cafe_americano['Americano Price1'] = cafe_americano['Americano'].apply(lambda x:x.strip())
<ipython-input-49-8244facd69a7>:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cafe_americano['Americano Price2'] = cafe_americano['Americano Price1'].apply(lambda x:x.split(' ')[-1])
<ipython-input-49-8244facd69a7>:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_index

ValueError: invalid literal for int() with base 10: '변동가격(업주문의)'